In [1]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

import croniter
import pandas as pd
import datetime
from datetime import timedelta
import re

# Croniter

In [498]:
now = datetime.datetime.now()
iter = croniter.croniter('55 23 * * Sat,Mon-Fri 55', pd.Timestamp('2023-08-25'))
iter = croniter.croniter('55 23 * * Sat 55', pd.Timestamp('2023-08-25'))
iter = croniter.croniter('* * * * * */3', pd.Timestamp('2023-08-25 12:00'))

print(pd.Timestamp(iter.get_prev(), unit='s'))
print('- - - -')
for _ in range(10):
    print(pd.Timestamp(iter.get_next(), unit='s'))

2023-08-25 11:59:57
- - - -
2023-08-25 12:00:00
2023-08-25 12:00:03
2023-08-25 12:00:06
2023-08-25 12:00:09
2023-08-25 12:00:12
2023-08-25 12:00:15
2023-08-25 12:00:18
2023-08-25 12:00:21
2023-08-25 12:00:24
2023-08-25 12:00:27


In [6]:
# # cmp = pd.Timedelta("6d23h55min").components
# cmp = pd.Timedelta("5Min").components
# sched = f"{cmp.minutes} {cmp.hours} */{cmp.days} * * {cmp.seconds}"
# iter = croniter.croniter(sched, pd.Timestamp('2023-08-25 12:00'))

# pd.Timestamp(iter.get_prev(), unit='s'), pd.Timestamp(iter.get_next(), unit='s')

In [ ]:
from dateutil.parser import parser
from dateutil.rrule import rrulestr
# list(rrulestr("""RRULE:FREQ=DAILY;INTERVAL=10"""))
list(rrulestr('FREQ=WEEKLY;INTERVAL=1;BYHOUR=23;BYMINUTE=45;BYSECOND=0;COUNT=5'))
list(rrulestr('FREQ=WEEKLY;INTERVAL=1;BYHOUR=23;BYMINUTE=45;BYSECOND=0;COUNT=5'))

# Spec parser

In [408]:
%load_ext autoreload
%autoreload 2
from qubx.core.helpers import BasicScheduler, process_schedule_spec
from qubx.utils.time import convert_seconds_to_str

In [411]:
specs = [
    '',
    'bar.1m: -5Sec',        # 5 sec before subsciption bar closing
    '1Min -5Sec',           # 5 sec before subsciption bar closing
    'bar:  5Sec',           # 5 sec after subsciption bar opened
    'bar:  +5Sec',          # 5 sec after subsciption bar opened
    'time: 23:59:00 @ Sat', # at 23:59:00 every Saturday
    'time: 23:59 @ Sat',    # at 23:59:00 every Saturday
    'time: 9:30:10',        # every day at 9:30:10
    '* * * * * 45',         # raw cron 
    'cron: * * * * * 45',   # raw cron witch implicit type
    '23:59:00 @ Sun',       # at 23:59:00 every Sunday
    '9:30:10',              # every day at 9:30:10
    '9:30 @ Mon-Fri',       # mon, ... fri at 9:30
    '9:30:22 @ Mon,Fri',    # mon and fri at 9:30:22
    '9:30 , 15:45 @ Mon-Fri', # mon, ... fri at 9:30 and 15:45
    '9:30, 15:45 @ Mon, Fri', # mon and fri at 9:30 and 15:45
    '9:30:10  15:45 @ Mon, Fri', # mon and fri at 9:30 and 15:45
    'calendar: 23:00 liquidation', # custom event
    '9:30 -1Min'            # time 
    '-5Sec',                # 5 sec before subsciption bar end
    '5Min -5Sec',           # 5 sec before every 5min interval ends
    '1w -1h',               # 1h before every week ends
    '4month -5hour',        # 5h before the end of every 4 months 
    '1week -3hours -10Mins',# 5h before the end of every 4 months 
    '2h30m -5Sec',          # 5 sec before every 2h and 30min interval ends
    '1w @ Sat',             # 5 sec before every 2h and 30min interval ends
]

In [415]:
for s in specs:
    print(s.ljust(30), process_schedule_spec(s))

                               []
bar.1m: -5Sec                  [{'type': 'bar', 'args': None, 'timeframe': '1m', 'delay': Timedelta('-1 days +23:59:55'), 'spec': '-5Sec'}]
1Min -5Sec                     [{'type': 'bar', 'args': None, 'timeframe': '1Min', 'delay': Timedelta('-1 days +23:59:55'), 'spec': '1Min -5Sec'}]
bar:  5Sec                     [{'type': 'bar', 'args': None, 'timeframe': None, 'delay': Timedelta('0 days 00:00:05'), 'spec': '5Sec'}]
bar:  +5Sec                    [{'type': 'bar', 'args': None, 'timeframe': None, 'delay': Timedelta('0 days 00:00:05'), 'spec': '+5Sec'}]
time: 23:59:00 @ Sat           [{'type': 'cron', 'args': '59 23 * * Sat', 'spec': '23:59:00 @ Sat'}]
time: 23:59 @ Sat              [{'type': 'cron', 'args': '59 23 * * Sat', 'spec': '23:59 @ Sat'}]
time: 9:30:10                  [{'type': 'cron', 'args': '30 9 * * * 10', 'spec': '9:30:10'}]
* * * * * 45                   [{'type': 'cron', 'args': '* * * * * 45', 'spec': '* * * * * 45'}]
cron: * * * *

In [414]:
if 0:
    for s in specs: 
        m = re.compile(
            r"((?P<type>[A-Za-z]+)(\.?(?P<timeframe>[0-9A-Za-z]+))?\ *:)?"
            r"\ *"
            r"((?P<spec>"
                r"(?P<time>((\d+:\d+(:\d+)?)\ *,?\ *)+)?"
                r"((\ *@\ *)(?P<by>([A-Za-z0-9-,\ ]+)))?"
                r"(("
                r'((?P<months>[-+]?\d+)(months|month|bm|mo))?'
                r'((?P<weeks>[-+]?\d+)(weeks|week|w))?'
                r'((?P<days>[-+]?\d+)(days|day|d))?'
                r'((?P<hours>[-+]?\d+)(hours|hour|h))?'
                r'((?P<minutes>[-+]?\d+)(mins|min|m))?'
                r'((?P<seconds>[-+]?\d+)(sec|s))?'
                r")(\ *)?)*"
                r".*"
            r"))?"

            , re.IGNORECASE
        ).match(s)
        print(s.ljust(30), '->', {k: v for k, v in m.groupdict().items() if v} if m else '?????')


# Scheduler tests

In [499]:
%load_ext autoreload
%autoreload 2
from qubx.core.helpers import BasicScheduler
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [457]:
import sched
import time as time_module

def myfunc(): 
    print(f"Trigger: {pd.Timestamp('now')}")

scheduler = sched.scheduler(time_module.time, time_module.sleep)
t1 = (pd.Timestamp('now', tz='UTC') + pd.Timedelta('3s')).round('3s').timestamp()
t2 = (pd.Timestamp('now', tz='UTC') + pd.Timedelta('5s')).round('5s').timestamp()
scheduler_e = scheduler.enterabs(t1, 1, myfunc, ())
scheduler_e = scheduler.enterabs(t2, 1, myfunc, ())

while (r:=scheduler.run(blocking=False)):
    time_module.sleep(max(r/5, 0.1))

Trigger: 2024-05-04 09:54:06.068088
Trigger: 2024-05-04 09:54:10.081555


In [500]:
time_now = lambda: pd.Timestamp('now', tz='UTC').as_unit('ns').asm8.item()

In [492]:
time_now()

1714807764627615000

In [520]:
bs = BasicScheduler(None, time_now)
bs.schedule_event('* * * * * */3', 'TEST')

In [522]:
bs.run()

2024-05-04 10:48:34.642 [ 🐞 ] Scheduled (TEST) task: next run at 2024-05-04 07:48:36
2024-05-04 10:48:34.643 [ ℹ️ ]  prev 2024-05-04 07:46:54 --> (2024-05-04 07:47:00) TEST 
2024-05-04 10:48:34.644 [ 🐞 ] Scheduled (TEST) task: next run at 2024-05-04 07:48:36
2024-05-04 10:48:36.060 [ ℹ️ ]  prev 2024-05-04 07:46:57 --> (2024-05-04 07:48:36) TEST 
2024-05-04 10:48:36.060 [ 🐞 ] Scheduled (TEST) task: next run at 2024-05-04 07:48:39
2024-05-04 10:48:36.060 [ ℹ️ ]  prev 2024-05-04 07:48:33 --> (2024-05-04 07:48:36) TEST 
2024-05-04 10:48:36.060 [ 🐞 ] Scheduled (TEST) task: next run at 2024-05-04 07:48:39
2024-05-04 10:48:39.013 [ ℹ️ ]  prev 2024-05-04 07:48:33 --> (2024-05-04 07:48:39) TEST 
2024-05-04 10:48:39.013 [ 🐞 ] Scheduled (TEST) task: next run at 2024-05-04 07:48:42
2024-05-04 10:48:39.013 [ ℹ️ ]  prev 2024-05-04 07:48:36 --> (2024-05-04 07:48:39) TEST 
2024-05-04 10:48:39.013 [ 🐞 ] Scheduled (TEST) task: next run at 2024-05-04 07:48:42


KeyboardInterrupt: 